<a href="https://colab.research.google.com/github/SivAsanjai/SivAsanjai/blob/main/DrugSpellCorrection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.3 MB/s eta 0:00:00


In [21]:
import csv
import nltk 
from nltk import ngrams
from collections import defaultdict, Counter
from spellchecker import SpellChecker

drug_dict = {}
with open('/content/drug_info_clean_1k.csv' , 'r') as file:
  csv_reader = csv.reader(file)
  header = next(csv_reader)
  for row in csv_reader:
    drug_name = row[header.index('description')].split(' ', 1)[0]
    drug_dict[drug_name.lower()] = row
n= 3
corpus = ' '.join(list(drug_dict.keys()))
tokens = nltk.word_tokenize(corpus)
ngrams_list = list(ngrams(tokens, n))

model  = defaultdict(Counter)
for ngram in ngrams_list:
  ngram_till_n_1 = tuple(ngram[:-1])
  last_word = ngram[-1]
  model[ngram_till_n_1][last_word] += 1

spell = SpellChecker(language=None, case_sensitive=True)
spell.word_frequency.load_words(drug_dict.keys())

def correct_spelling(word):
  return spell.correction(word)

print(correct_spelling('Yevozan'))

yevozen


In [27]:
import csv
from textblob import TextBlob

# Parse the CSV
drug_dict = {}
with open ('/content/drug_info_clean_1k.csv', 'r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    for row in csv_reader:
        drug_name = row[header.index('description')].split(' ', 1)[0]
        drug_dict[drug_name.lower()] = row
n= 3
corpus = ' '.join(list(drug_dict.keys()))
tokens = nltk.word_tokenize(corpus)
ngrams_list = list(ngrams(tokens, n))

model  = defaultdict(Counter)
for ngram in ngrams_list:
  ngram_till_n_1 = tuple(ngram[:-1])
  last_word = ngram[-1]
  model[ngram_till_n_1][last_word] += 1

spell = SpellChecker(language=None, case_sensitive=True)
spell.word_frequency.load_words(drug_dict.keys())

def correct_spelling(word):
    blob = TextBlob(word)
    correct_word = blob.correct()
    return correct_word if correct_word in drug_dict else word 

# Test spelling correction
print(correct_spelling('Yevizan'))


Yevizan


In [37]:
def get_ngrams(word, n = 3):
  return set(ngrams(word, n))
def jaccard_similarity(set1, set2):
  intersection = set1.intersection(set2)
  union = set1.union(set2)
  return len(intersection)/len(union)
def correct_spelling(word):
  word_ngrams = get_ngrams(word)
  similarities = [(drug, jaccard_similarity(word_ngrams,get_ngrams(drug))) for drug in drug_dict.keys()]
  most_similar = max(similarities, key = lambda x: x[1])
  return most_similar[0] if most_similar[0] in drug_dict.keys() else None


In [41]:
print(correct_spelling('Yonikest'))

yonikast


<ipython-input-44-f6073940e26d>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sequences = array(sequences)


IndexError: ignored

In [51]:
import numpy as np
from numpy import array 
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential 
from keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split


In [53]:
drug_names = list(drug_dict.keys())
chars = sorted(list(set(''.join(drug_names))))
mapping = dict((c, i) for i,c in enumerate(chars))
sequences = []
for name in drug_names:
  encoded_seq = [mapping[char] for char in name]
  for i in range(1, len(encoded_seq)):
    sequences.append(encoded_seq[:i+1])

sequences = pad_sequences(sequences)
x,y = sequences[:,:-1], sequences[:, -1]
x = x / float(len(mapping))
y = to_categorical(y, num_classes = len(mapping))

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state = 1)

model = Sequential()
model.add(LSTM(75, input_shape = (x.shape[1],1)))
model.add(Dense(len(mapping),activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer= 'adam', metrics = ['accuracy'])

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1],1)

model.fit(x_train, y_train, epochs = 100, verbose = 2, validation_data = (x_test, y_test))

model.save('drug_lstm1.h5')


Epoch 1/100
97/97 - 4s - loss: 3.0889 - accuracy: 0.1198 - val_loss: 2.9244 - val_accuracy: 0.1358 - 4s/epoch - 45ms/step
Epoch 2/100
97/97 - 1s - loss: 2.9637 - accuracy: 0.1240 - val_loss: 2.9141 - val_accuracy: 0.1268 - 930ms/epoch - 10ms/step
Epoch 3/100
97/97 - 1s - loss: 2.9499 - accuracy: 0.1250 - val_loss: 2.8966 - val_accuracy: 0.1255 - 971ms/epoch - 10ms/step
Epoch 4/100
97/97 - 1s - loss: 2.9249 - accuracy: 0.1292 - val_loss: 2.8838 - val_accuracy: 0.1320 - 944ms/epoch - 10ms/step
Epoch 5/100
97/97 - 1s - loss: 2.9153 - accuracy: 0.1279 - val_loss: 2.8813 - val_accuracy: 0.1384 - 960ms/epoch - 10ms/step
Epoch 6/100
97/97 - 1s - loss: 2.9055 - accuracy: 0.1253 - val_loss: 2.8537 - val_accuracy: 0.1345 - 935ms/epoch - 10ms/step
Epoch 7/100
97/97 - 1s - loss: 2.8890 - accuracy: 0.1321 - val_loss: 2.8547 - val_accuracy: 0.1371 - 1s/epoch - 10ms/step
Epoch 8/100
97/97 - 1s - loss: 2.8708 - accuracy: 0.1354 - val_loss: 2.8381 - val_accuracy: 0.1358 - 1s/epoch - 12ms/step
Epoch 9/1

In [62]:
def correct_spelling(model, mapping, input_word):
  encoded_input = [mapping.get(char, 0) for char in input_word]
  encoded_input = pad_sequences([encoded_input], maxlen= x.shape[1], truncating = 'pre') / float(len(mapping))
  encoded_input = encoded_input.reshape(encoded_input.shape[0],encoded_input.shape[1], 1)
  prediction = model.predict(encoded_input, verbose=0)
  index = np.argmax(prediction)
  out_word = ''
  for char, idx in mapping.items():
    if idx == index:
      out_word += char
      break
  return out_word

In [66]:
print(correct_spelling(model, mapping, 'Yevoze'))

l


In [68]:
def correct_spelling_beam_search(model, mapping, input_word, k=3, sequence_max_len=100):
    input_sequence = [mapping.get(char, 0) for char in input_word]
    input_sequence = pad_sequences([input_sequence], maxlen=sequence_max_len, truncating='pre') / float(len(mapping))
    input_sequence = input_sequence.reshape(input_sequence.shape[0], input_sequence.shape[1], 1)

    # Initialize beam search
    sequences = [[list(), 0.0]]

    # Loop over sequence length
    for _ in range(sequence_max_len):
        all_candidates = list()
        # Loop over each sequence in the beam
        for i in range(len(sequences)):
            seq, score = sequences[i]
            yhat = model.predict([seq])[0]
            # Loop over each possible next token
            for j in range(len(yhat)):
                # Create a new candidate sequence
                candidate = [seq + [j], score - np.log(yhat[j])]
                all_candidates.append(candidate)
        # Rank all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # Select k best
        sequences = ordered[:k]
    best_sequence = sequences[0]
    out_word = [x for x in best_sequence[0]]
    return ''.join(out_word)

# Testing the prediction
print(correct_spelling_beam_search(model, mapping, 'Yevozen'))


KeyError: ignored

In [93]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def char_ngrams(text, n):
    return [text[i:i+n] for i in range(len(text)-n+1)]

def char_vector(text, n, vocabulary):
    text_ngrams = char_ngrams(text, n)
    return np.array([text_ngrams.count(i) for i in vocabulary])

def cosine_similarity_char_ngram(text1, text2, n):
    ngrams_text1 = char_ngrams(text1, n)
    ngrams_text2 = char_ngrams(text2, n)
    vocabulary = list(set(ngrams_text1 + ngrams_text2))
    vector1 = char_vector(text1, n, vocabulary)
    vector2 = char_vector(text2, n, vocabulary)
    if norm(vector1) != 0 and norm(vector2) != 0:
        cos_sim = dot(vector1, vector2)/(norm(vector1)*norm(vector2))
    else:
        cos_sim = 0
    return cos_sim


def correct_spelling_cosine(word, drug_dict, ngram=2):
    similarities = [(cosine_similarity_char_ngram(word, key, ngram), key) for key in drug_dict.keys()]
    similarities.sort(reverse=True)
    return [similarities[i][1] for i in range(0,len(similarities))]




In [97]:
print(correct_spelling_cosine('pekadin', drug_dict))

['dekadin', 'dikacin', 'yacadine', 'dinosin', 'yinka', 'yekin', 'dilkab', 'diepin', 'yukacin', 'divaine', 'dicline', 'yoridine', 'domtidin', 'dizapear', 'dicolead', 'diapyrin', 'yukacin-o', 'dil', 'davaindia', 'yzin', 'yaka', 'digo', 'dice', 'additionally,', 'yzine', 'yucin', 'yosin', 'yline', 'yezdi', 'dutin', 'dupin', 'dppin', 'dmine', 'divon', 'dival', 'ditim', 'disal', 'diroz', 'diran', 'dipod', 'difen', 'derek', 'depel', 'yoxtin', 'yoflin', 'ycline', 'yasmin', 'yamini', 'yamika', 'yadrox', 'dxfine', 'duowin', 'duopep', 'divsys', 'divcet', 'dithro', 'disera', 'dipkem', 'diopan', 'diolof', 'dioflo', 'dilzem', 'diflox', 'diflam', 'dicnos', 'dicmol', 'dicmak', 'diclom', 'deswin', 'dellin', 'yoritad', 'yorimin', 'yoridiv', 'yevomin', 'yetoxin', 'dynocad', 'drozine', 'dpazine', 'douxper', 'docitin', 'dmentin', 'dixovan', 'divorel', 'divazen', 'divasag', 'divacof', 'dispasm', 'disoval', 'dilocus', 'dilnorm', 'difsoon', 'difisal', 'difenic', 'diecold', 'dicotan', 'diclove', 'dicipam', 'di